# 데이터 합치기
반입량 계산, 열 변경

In [2]:
import pandas as pd

df_price = pd.read_csv('data/CabbagePrice.CSV')
df_intake = pd.read_csv('data/CabbageIntake.CSV')

# DATE 기준으로 merge
merged_df = pd.merge(df_price, df_intake[['DATE', 'intake']], on='DATE', how='left')

# 등급 비율 정의
grade_ratio = {
  '특': 0.05,
  '상': 0.35,
  '보통': 0.40,
  '하': 0.20
}

# 비율 적용해서 반입량 계산
def calculate_amount(row):
  grade = row['rate']
  total_amount = 0 if pd.isna(row['intake']) else row['intake']
  return int(total_amount * grade_ratio.get(grade, 0))

merged_df['intake'] = merged_df.apply(calculate_amount, axis=1)


# 특 데이터 필터링
df_price_special = merged_df[merged_df['rate'] == '특'].drop(columns=['rate', 'unit'])

# 상 데이터 필터링
df_price_high = merged_df[merged_df['rate'] == '상'].drop(columns=['rate', 'unit'])

# 열 순서 변경 year 년, month 월, date 일, unit 단위, intake 반입량, avg_price 평균가격
df_price_special = df_price_special[['DATE', 'intake', 'avg_price', ]]
df_price_high = df_price_high[['DATE', 'intake', 'avg_price', ]]

# CSV 파일로 저장
df_price_special.to_csv('data/CabbageSpecial.csv', index=False)
df_price_high.to_csv('data/CabbageHigh.csv', index=False)

print("Data saved successfully.")

Data saved successfully.


# 상 가격 분석

In [ ]:
import pandas as pd
df_high = pd.read_csv('data/CabbageHigh.csv', parse_dates=['year', 'month', 'date'])

df_high["prev_price"] = df_high["avg_price"].shift(1)
df_high["rolling_avg_3"] = df_high["avg_price"].shift(1).rolling(3).mean()
df_high["inflow_change"] = df_high["intake"].pct_change()

C:\Users\cksdn\AppData\Local\Temp\ipykernel_20036\3921607035.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_high = pd.read_csv('data/CabbageHigh.csv', parse_dates=['year', 'month', 'date'])
C:\Users\cksdn\AppData\Local\Temp\ipykernel_20036\3921607035.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_high = pd.read_csv('data/CabbageHigh.csv', parse_dates=['year', 'month', 'date'])


In [7]:
df_high

,year,month,date,unit,intake,avg_price,prev_price,rolling_avg_3,inflow_change
0,2025-01-01,4,2,10,0,14170,NaN,NaN,NaN
1,2025-01-01,4,1,10,137,13460,14170.0,NaN,inf
2,2025-01-01,3,31,10,132,14063,13460.0,NaN,-0.036496
3,2025-01-01,3,30,10,0,0,14063.0,13897.666667,-1.000000
4,2025-01-01,3,29,10,101,11024,0.0,9174.333333,inf
...,...,...,...,...,...,...,...,...,...
4586,2012-01-01,1,5,10,96,2656,2623.0,2491.000000,-0.219512
4587,2012-01-01,1,4,10,0,2921,2656.0,2561.333333,-1.000000
4588,2012-01-01,1,3,10,0,2613,2921.0,2733.333333,NaN
4589,2012-01-01,1,2,10,0,2323,2613.0,2730.000000,NaN


# DB 저장용 일별 데이터
일별

In [3]:
import pandas as pd

# 데이터 불러오기
df_high_filtered = pd.read_csv('data/CabbageHigh.csv')

# DATE 컬럼을 datetime 형식으로 변환
df_high_filtered['DATE'] = pd.to_datetime(df_high_filtered['DATE'])

# 가장 최신 날짜를 latest_date로 지정
latest_date = df_high_filtered['DATE'].max()

# 평균 가격이 0이 아닌 데이터만 남기고, 날짜 역순 정렬
df_high_filtered = df_high_filtered[df_high_filtered['avg_price'] != 0]
df_high_filtered = df_high_filtered.sort_values(by='DATE', ascending=True).reset_index(drop=True)

# 어제 대비 가격 차이 계산
df_high_filtered['gap'] = df_high_filtered['avg_price'].diff().fillna(0).astype(int)

# 12개월 전 날짜 계산 후 필터링
cutoff_date = latest_date - pd.DateOffset(months=12)
df_high_filtered = df_high_filtered[df_high_filtered['DATE'] >= cutoff_date]

# 날짜 정보 분리
df_high_filtered['year'] = df_high_filtered['DATE'].dt.year
df_high_filtered['month'] = df_high_filtered['DATE'].dt.month
df_high_filtered['day'] = df_high_filtered['DATE'].dt.day

# 필요한 컬럼 순서대로 정리
df_high_filtered = df_high_filtered[['year', 'month', 'day', 'intake', 'avg_price', 'gap']]

# 결과 저장
df_high_filtered.to_csv('store/CabbageHigh.csv', index=False)

print("Filtered data saved successfully.")

Filtered data saved successfully.


In [4]:
import pandas as pd

# 데이터 불러오기
df_spe_filtered = pd.read_csv('data/CabbageSpecial.csv')

# DATE 컬럼을 datetime 형식으로 변환
df_spe_filtered['DATE'] = pd.to_datetime(df_spe_filtered['DATE'])

# 가장 최신 날짜를 latest_date로 지정
latest_date = df_spe_filtered['DATE'].max()

# 평균 가격이 0이 아닌 데이터만 남기고, 날짜 역순 정렬
df_spe_filtered = df_spe_filtered[df_spe_filtered['avg_price'] != 0]
df_spe_filtered = df_spe_filtered.sort_values(by='DATE', ascending=True).reset_index(drop=True)

# 어제 대비 가격 차이 계산
df_spe_filtered['gap'] = df_spe_filtered['avg_price'].diff().fillna(0).astype(int)

# 12개월 전 날짜 계산 후 필터링
cutoff_date = latest_date - pd.DateOffset(months=12)
df_spe_filtered = df_spe_filtered[df_spe_filtered['DATE'] >= cutoff_date]

# 날짜 정보 분리
df_spe_filtered['year'] = df_spe_filtered['DATE'].dt.year
df_spe_filtered['month'] = df_spe_filtered['DATE'].dt.month
df_spe_filtered['day'] = df_spe_filtered['DATE'].dt.day

# 필요한 컬럼 순서대로 정리
df_spe_filtered = df_spe_filtered[['year', 'month', 'day', 'intake', 'avg_price', 'gap']]

# 결과 저장
df_spe_filtered.to_csv('store/CabbageSpecial.csv', index=False)

print("Filtered data saved successfully.")

Filtered data saved successfully.
